In [34]:
# import needed packages
import csv
import sqlite3
import pandas as pd

In [35]:
#read raw data CSV into ABC_data_table in ABC.db
pd.read_csv('ABCLicenseData.csv', header=0, nrows=None, chunksize=None, index_col=0, na_values=['n/a']).to_sql('ABC_data_table', sqlite3.connect("ABC.db"), if_exists = "replace")

In [38]:
con = sqlite3.connect("ABC.db")
#display all columns
pd.options.display.max_columns = 500
#use sql query to read database table into pandas dataframe object, ommitting null IssueDate's and DISTRICT's
data_df = pd.read_sql_query("SELECT * FROM ABC_data_table WHERE IssueDate IS NOT NULL AND DISTRICT IS NOT NULL" , con, parse_dates=True,)
#parse year from datatime field
data_df['IssueDate'] = pd.to_datetime(data_df['IssueDate'])
#convert year and district to int datatype
data_df['Year'] = data_df['IssueDate'].dt.year.astype('int')
data_df['DISTRICT'] = data_df['DISTRICT'].astype('int')
#read cleaned up data back into database table
data_df.to_sql('ABC_data_table', con, schema=None, if_exists='replace', index=False, index_label=None)
#display first five records for review
data_df.head()

,CodesAndRegsLicenseDataID,LicenseID,Description,LicenseName,Location,StreetNumber,Prefix,StreetName,Suffix,Longitude,Latitude,ExpirationDate,SubType,SubDescription,IssueDate,Zip,DISTRICT,NEIGHBORHOOD,ZONING,EndorsementType,EndorsementTypeDescription,EndorsementStatusDescription,EndorsementIssuedDate,AGENCY,GPSX,GPSY,Year
0,1,5677,Alcoholic Beverage Annual,TUMBLEWEED SOUTHWEST GRILL,4255 OUTER LOOP,4255,,OUTER,LOOP,-85.682816,38.138772,2018-08-31 00:00:00,RST,Restaurant / Cafe,2004-08-17 10:14:00,40219,24,None,C1,RLD,NQ2 Retail Drink (Restaurant),ACTIVE,None,Codes and Regulations,1228336.0,235821.0,2004
1,2,5677,Alcoholic Beverage Annual,TUMBLEWEED SOUTHWEST GRILL,4255 OUTER LOOP,4255,,OUTER,LOOP,-85.682816,38.138772,2018-08-31 00:00:00,RST,Restaurant / Cafe,2004-08-17 10:14:00,40219,24,None,C1,SD,Sunday Drink,ACTIVE,None,Codes and Regulations,1228336.0,235821.0,2004
2,3,5679,Alcoholic Beverage Annual,TUMBLEWEED SOUTHWEST GRILL,3550 SPRINGHURST COMMONS DR,3550,,SPRINGHURST COMMONS,DR,-85.561421,38.297505,2018-08-31 00:00:00,RST,Restaurant / Cafe,2004-08-03 16:22:00,40241,17,None,C1,RLD,NQ2 Retail Drink (Restaurant),ACTIVE,None,Codes and Regulations,1264070.0,293100.0,2004
3,4,5679,Alcoholic Beverage Annual,TUMBLEWEED SOUTHWEST GRILL,3550 SPRINGHURST COMMONS DR,3550,,SPRINGHURST COMMONS,DR,-85.561421,38.297505,2018-08-31 00:00:00,RST,Restaurant / Cafe,2004-08-03 16:22:00,40241,17,None,C1,SD,Sunday Drink,ACTIVE,None,Codes and Regulations,1264070.0,293100.0,2004
4,5,4642,Alcoholic Beverage Annual,CARDINAL HALL OF FAME CAFE,2745 CRITTENDEN DR,2745,,CRITTENDEN,DR,-85.748965,38.203738,2019-05-31 00:00:00,RST,Restaurant / Cafe,2004-05-25 08:47:00,40209,15,FAIRGROUNDS,M2,SBL,Supplemental Bar,ACTIVE,None,Codes and Regulations,1209698.0,259779.0,2004


In [74]:
con = sqlite3.connect("ABC.db")
pd.options.display.max_columns = 500
hist_df = pd.read_sql_query('''
SELECT DISTRICT as District, COUNT(LicenseName) AS '2012-2016 Total'
FROM ABC_data_table
WHERE Year BETWEEN 2012 AND 2016
GROUP BY DISTRICT
''', con)
hist_df['2012-2016 Average'] = hist_df['2012-2016 Total']/5
hist_df['2017 Total'] = pd.read_sql_query('''
SELECT COUNT(LicenseName)
FROM ABC_data_table
WHERE Year = 2017
GROUP BY DISTRICT
''', con)
display(hist_df)

,District,2012-2016 Total,2012-2016 Average,2017 Total
0,1,13,2.6,34
1,2,28,5.6,25
2,3,25,5.0,46
3,4,287,57.4,193
4,5,29,5.8,58
5,6,98,19.6,83
6,7,33,6.6,10
7,8,141,28.2,66
8,9,69,13.8,78
9,10,100,20.0,71


In [43]:
con = sqlite3.connect("ABC.db")
pd.options.display.max_columns = 500
data_df = pd.read_sql_query('''
SELECT COUNT(DISTINCT Zip), COUNT(DISTINCT DISTRICT) 
FROM ABC_data_table
''', con)
display(data_df)

,COUNT(DISTINCT Zip),COUNT(DISTINCT DISTRICT)
0,34,26


In [44]:
con = sqlite3.connect("ABC.db")
pd.options.display.max_columns = 500
data_df = pd.read_sql_query('''
SELECT LicenseName, IssueDate, Zip, DISTRICT 
FROM ABC_data_table
ORDER BY IssueDate
''', con)
display(data_df)

,LicenseName,IssueDate,Zip,DISTRICT
0,FULL SWING INDOOR GOLF,2004-02-09 14:21:00,40219,21
1,OKOLONA POST #8639,2004-02-09 15:43:00,40219,24
2,OKOLONA POST #8639,2004-02-09 15:43:00,40219,24
3,ST IGNATIUS BOOSTER CLUB,2004-02-09 16:24:00,40219,2
4,MT CARMEL MEN'S CLUB,2004-02-09 17:56:00,40214,13
5,CHURCH OF THE GUARDIAN ANGELS,2004-02-09 18:01:00,40219,21
6,IROQUOIS HILL POST #6182 VFW,2004-02-10 08:29:00,40215,15
7,IROQUOIS HILL POST #6182 VFW,2004-02-10 08:29:00,40215,15
8,IROQUOIS HILL POST #6182 VFW,2004-02-10 08:29:00,40215,15
9,EL TARASCO I,2004-02-10 08:55:00,40214,25
